In [ ]:
!pip install streamlit

In [ ]:
!pip install --upgrade plotly


In [3]:
import plotly
import pandas as pd
import numpy as np
import streamlit as st
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [4]:
import pickle

### Load in the Dataset

In [12]:
df = pd.read_csv('flights_subset.csv')
df

,Unnamed: 0,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,f61355d2300d1e7d070752f69793de10,2022-04-16,2022-04-19,DFW,SFO,WAA3PKEN,PT7H11M,0,False,...,1650392640||1650414000,2022-04-19T13:24:00.000-05:00||2022-04-19T17:2...,IAH||SFO,DFW||IAH,United||United,UA||UA,Airbus A319||Boeing 737-900,4500||15300,233||1641,coach||coach
19996,19996,d58b67c57020ff78f8bd3a691e16d9a3,2022-04-16,2022-04-19,DFW,SFO,QH3OAJBN,PT7H19M,0,True,...,1650389880||1650400440,2022-04-19T10:38:00.000-07:00||2022-04-19T13:3...,SEA||SFO,DFW||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737-800||Boeing 737 MAX 9,15780||7800,1663||679,coach||coach
19997,19997,dfda3baa34e1b0e12839040c14370663,2022-04-16,2022-04-19,DFW,SFO,QH3OAJBN,PT8H10M,0,True,...,1650389880||1650403500,2022-04-19T10:38:00.000-07:00||2022-04-19T14:2...,SEA||SFO,DFW||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737-800||Airbus A320,15780||7500,1663||679,coach||coach
19998,19998,aa6e33e0918799d8d1d6347890460a24,2022-04-16,2022-04-19,DFW,SFO,WAA3PKEN,PT8H27M,0,False,...,1650397260||1650418980,2022-04-19T14:41:00.000-05:00||2022-04-19T18:4...,ORD||SFO,DFW||ORD,United||United,UA||UA,Airbus A320||Boeing 757-300,8700||16620,799||1847,coach||coach


In [6]:
df.columns

Index(['Unnamed: 0', 'legId', 'searchDate', 'flightDate', 'startingAirport',
       'destinationAirport', 'fareBasisCode', 'travelDuration', 'elapsedDays',
       'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'totalFare',
       'seatsRemaining', 'totalTravelDistance',
       'segmentsDepartureTimeEpochSeconds', 'segmentsDepartureTimeRaw',
       'segmentsArrivalTimeEpochSeconds', 'segmentsArrivalTimeRaw',
       'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode',
       'segmentsAirlineName', 'segmentsAirlineCode',
       'segmentsEquipmentDescription', 'segmentsDurationInSeconds',
       'segmentsDistance', 'segmentsCabinCode'],
      dtype='object')

### Pre-processing the data

In [14]:
np.unique(df.startingAirport)

array(['ATL', 'BOS', 'CLT', 'DEN', 'DFW', 'DTW', 'EWR', 'IAD', 'JFK',
       'LAX', 'LGA', 'MIA', 'OAK', 'ORD', 'PHL', 'SFO'], dtype=object)

In [15]:
np.unique(df.destinationAirport)

array(['ATL', 'BOS', 'CLT', 'DEN', 'DFW', 'DTW', 'EWR', 'IAD', 'JFK',
       'LAX', 'LGA', 'MIA', 'OAK', 'ORD', 'PHL', 'SFO'], dtype=object)

In [17]:
np.unique(processed_data.AirlineName)

array(['Alaska Airlines', 'American Airlines', 'Boutique Air', 'Cape Air',
       'Contour Airlines', 'Delta', 'Frontier Airlines',
       'JetBlue Airways', 'Key Lime Air', 'Southern Airways Express',
       'Spirit Airlines', 'Sun Country Airlines', 'United'], dtype=object)

In [18]:
def one_hot_encode_data(df, columns):
    return pd.get_dummies(df, columns=columns)

# Example columns to encode
columns_to_encode = ['startingAirport', 'destinationAirport', 'AirlineName']
one_hot_encode_data(processed_data, columns_to_encode)

,isBasicEconomy,isRefundable,isNonStop,days_before_flight,day,num_legs,all_same,departure_hour,departure_dow_idx,totalFare,...,AirlineName_Cape Air,AirlineName_Contour Airlines,AirlineName_Delta,AirlineName_Frontier Airlines,AirlineName_JetBlue Airways,AirlineName_Key Lime Air,AirlineName_Southern Airways Express,AirlineName_Spirit Airlines,AirlineName_Sun Country Airlines,AirlineName_United
0,0,0,1,1,17,1,1,12,6,248.60,...,0,0,1,0,0,0,0,0,0,0
1,0,0,1,1,17,1,1,6,6,248.60,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,1,17,1,1,11,6,248.60,...,0,0,1,0,0,0,0,0,0,0
3,0,0,1,1,17,1,1,13,6,248.60,...,0,0,1,0,0,0,0,0,0,0
4,0,0,1,1,17,1,1,9,6,248.60,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,3,19,2,0,15,1,262.61,...,0,0,0,0,0,0,0,0,0,1
19996,1,0,0,3,19,2,0,11,1,262.61,...,0,0,0,0,0,0,0,0,0,0
19997,1,0,0,3,19,2,0,12,1,262.61,...,0,0,0,0,0,0,0,0,0,0
19998,0,0,0,3,19,2,0,16,1,262.61,...,0,0,0,0,0,0,0,0,0,1


In [19]:
# Airports and Airlines Encoding
airports = ['ATL', 'BOS', 'CLT', 'DEN', 'DFW', 'DTW', 'EWR', 'IAD', 'JFK',
            'LAX', 'LGA', 'MIA', 'OAK', 'ORD', 'PHL', 'SFO']
airlines = ['Alaska Airlines', 'American Airlines', 'Boutique Air', 'Cape Air',
            'Contour Airlines', 'Delta', 'Frontier Airlines', 'JetBlue Airways',
            'Key Lime Air', 'Southern Airways Express', 'Spirit Airlines', 
            'Sun Country Airlines', 'United']

airport_index = {airport: i for i, airport in enumerate(airports)}
airline_index = {airline: i for i, airline in enumerate(airlines)}
airport_index

{'ATL': 0,
 'BOS': 1,
 'CLT': 2,
 'DEN': 3,
 'DFW': 4,
 'DTW': 5,
 'EWR': 6,
 'IAD': 7,
 'JFK': 8,
 'LAX': 9,
 'LGA': 10,
 'MIA': 11,
 'OAK': 12,
 'ORD': 13,
 'PHL': 14,
 'SFO': 15}

In [48]:
airline_index

{'Alaska Airlines': 0,
 'American Airlines': 1,
 'Boutique Air': 2,
 'Cape Air': 3,
 'Contour Airlines': 4,
 'Delta': 5,
 'Frontier Airlines': 6,
 'JetBlue Airways': 7,
 'Key Lime Air': 8,
 'Southern Airways Express': 9,
 'Spirit Airlines': 10,
 'Sun Country Airlines': 11,
 'United': 12}

In [60]:
import pandas as pd
from datetime import datetime

def select_columns_for_model(df):
    model_columns = ['isBasicEconomy', 'isRefundable', 'isNonStop', 'days_before_flight',
                     'day', 'num_legs', 'all_same', 'departure_hour', 
                     'departure_dow_idx', 'startingAirport', 'destinationAirport', 'AirlineName']
    return df[model_columns]

def process_dates(df):
    df['searchDate'] = pd.to_datetime(df['searchDate'])
    df['flightDate'] = pd.to_datetime(df['flightDate'])
    df['days_before_flight'] = (df['flightDate'] - df['searchDate']).dt.days
    df['day'] = df['flightDate'].dt.day
    df['departure_dow_idx'] = df['flightDate'].dt.dayofweek
    return df

def extract_departure_hour(df):
    df['departure_hour'] = df['segmentsDepartureTimeRaw'].apply(
        lambda x: pd.to_datetime(x.split('||')[1] if '||' in x else x.split('||')[0]).hour
    )
    return df

def count_segments_and_check_airline(df):
    df['num_legs'] = df['segmentsArrivalAirportCode'].apply(lambda x: len(str(x).split('||')))
    df['all_same'] = df['segmentsAirlineName'].apply(lambda x: '1' if '||' not in x else '0')
    return df

def preprocess_airline_names(df):
    # Split the 'segmentsAirlineName' entries and keep only the first airline
    df['AirlineName'] = df['segmentsAirlineName'].apply(lambda x: x.split('||')[0])
    return df


def convert_boolean_fields(df):
    df['isBasicEconomy'] = df['isBasicEconomy'].astype(int)
    df['isRefundable'] = df['isRefundable'].astype(int)
    df['isNonStop'] = df['isNonStop'].astype(int)
    return df


def preprocess_data(df):
    df = process_dates(df)
    df = extract_departure_hour(df)
    df = count_segments_and_check_airline(df)
#     df = handle_distance(df)
    df = preprocess_airline_names(df)
    df = convert_boolean_fields(df)
    df = select_columns_for_model(df)
    return df


def label_encode_data(df, columns):
    encoder = LabelEncoder()
    for column in columns:
        df[column] = encoder.fit_transform(df[column].astype(str))
    return df

# Example columns to encode
# columns_to_encode = ['departure_dow_idx', 'startingAirport', 'destinationAirport']
# processed_data = label_encode_data(processed_data, columns_to_encode)





#pre-processed the columns
data = pd.read_csv("flights_subset.csv")
processed_data = preprocess_data(data)
processed_data['all_same'] = processed_data['all_same'].astype(int) # Convert 'all_same' from string type '1', '0' to integer type 1, 0
processed_data['totalFare'] = df['totalFare']
processed_data

/Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,isBasicEconomy,isRefundable,isNonStop,days_before_flight,day,num_legs,all_same,departure_hour,departure_dow_idx,startingAirport,destinationAirport,AirlineName,totalFare
0,0,0,1,1,17,1,1,12,6,ATL,BOS,Delta,248.60
1,0,0,1,1,17,1,1,6,6,ATL,BOS,Delta,248.60
2,0,0,1,1,17,1,1,11,6,ATL,BOS,Delta,248.60
3,0,0,1,1,17,1,1,13,6,ATL,BOS,Delta,248.60
4,0,0,1,1,17,1,1,9,6,ATL,BOS,Delta,248.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,3,19,2,0,15,1,DFW,SFO,United,262.61
19996,1,0,0,3,19,2,0,11,1,DFW,SFO,Alaska Airlines,262.61
19997,1,0,0,3,19,2,0,12,1,DFW,SFO,Alaska Airlines,262.61
19998,0,0,0,3,19,2,0,16,1,DFW,SFO,United,262.61


In [61]:
np.unique(processed_data.AirlineName)

array(['Alaska Airlines', 'American Airlines', 'Boutique Air', 'Cape Air',
       'Contour Airlines', 'Delta', 'Frontier Airlines',
       'JetBlue Airways', 'Key Lime Air', 'Southern Airways Express',
       'Spirit Airlines', 'Sun Country Airlines', 'United'], dtype=object)

In [62]:
# Applying the encoding using the map function
processed_data['startingAirport_encoded'] = processed_data['startingAirport'].map(airport_index)
processed_data['destinationAirport_encoded'] = processed_data['destinationAirport'].map(airport_index)
processed_data['segmentsAirlineName_encoded'] = processed_data['AirlineName'].map(airline_index)
processed_data

/Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,isBasicEconomy,isRefundable,isNonStop,days_before_flight,day,num_legs,all_same,departure_hour,departure_dow_idx,startingAirport,destinationAirport,AirlineName,totalFare,startingAirport_encoded,destinationAirport_encoded,segmentsAirlineName_encoded
0,0,0,1,1,17,1,1,12,6,ATL,BOS,Delta,248.60,0,1,5
1,0,0,1,1,17,1,1,6,6,ATL,BOS,Delta,248.60,0,1,5
2,0,0,1,1,17,1,1,11,6,ATL,BOS,Delta,248.60,0,1,5
3,0,0,1,1,17,1,1,13,6,ATL,BOS,Delta,248.60,0,1,5
4,0,0,1,1,17,1,1,9,6,ATL,BOS,Delta,248.60,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,3,19,2,0,15,1,DFW,SFO,United,262.61,4,15,12
19996,1,0,0,3,19,2,0,11,1,DFW,SFO,Alaska Airlines,262.61,4,15,0
19997,1,0,0,3,19,2,0,12,1,DFW,SFO,Alaska Airlines,262.61,4,15,0
19998,0,0,0,3,19,2,0,16,1,DFW,SFO,United,262.61,4,15,12


In [63]:
np.unique(processed_data.segmentsAirlineName_encoded)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [64]:
processed_data.drop(['startingAirport', 'destinationAirport', 'AirlineName'], axis=1, inplace=True)


/Users/jordan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [65]:
processed_data

,isBasicEconomy,isRefundable,isNonStop,days_before_flight,day,num_legs,all_same,departure_hour,departure_dow_idx,totalFare,startingAirport_encoded,destinationAirport_encoded,segmentsAirlineName_encoded
0,0,0,1,1,17,1,1,12,6,248.60,0,1,5
1,0,0,1,1,17,1,1,6,6,248.60,0,1,5
2,0,0,1,1,17,1,1,11,6,248.60,0,1,5
3,0,0,1,1,17,1,1,13,6,248.60,0,1,5
4,0,0,1,1,17,1,1,9,6,248.60,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,3,19,2,0,15,1,262.61,4,15,12
19996,1,0,0,3,19,2,0,11,1,262.61,4,15,0
19997,1,0,0,3,19,2,0,12,1,262.61,4,15,0
19998,0,0,0,3,19,2,0,16,1,262.61,4,15,12


## Train the Model

In [66]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Assuming 'processed_data' is your DataFrame and 'target' is your target column
X = processed_data.drop('totalFare', axis=1)  # Feature matrix
y = processed_data['totalFare']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [67]:
# For a regression problem
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

# For a classification problem (uncomment the following line and comment the regression line above)
# model = xgb.XGBClassifier(objective='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)

model.fit(X_train, y_train)

# Predicting the Test set results
y_pred = model.predict(X_test)


In [68]:
y_test

10650    305.61
2041     370.60
8668     287.58
1114     189.10
13902    371.60
          ...  
4073     646.60
7442     652.70
9999     322.60
1870     238.60
15196    361.60
Name: totalFare, Length: 4000, dtype: float64

## Evaluation

In [69]:
from sklearn.metrics import mean_squared_error, accuracy_score

# For regression
mse = mean_squared_error(y_test, y_pred)
print(f"RMSE: {mse**0.5}")

# For classification (uncomment for classification problems)
# accuracy = accuracy_score(y_test, (y_pred > 0.5))
# print(f"Accuracy: {accuracy}")


RMSE: 258.94872293238575


In [70]:
# Save the model
model.save_model("streamlit_xgboost_model.json")

In [59]:
import streamlit as st
import pandas as pd
import xgboost as xgb
import numpy as np

# Function to load your trained model
@st.cache(allow_output_mutation=True)
def load_model():
    model = xgb.XGBRegressor()
    model.load_model("streamlit_xgboost_model.json")  # Update path to your XGBoost model file
    return model

# Main application
def main():
    st.title("Flight Feature Prediction")

    # Collecting user inputs
    is_basic_economy = st.selectbox("Is Basic Economy?", [0, 1])
    is_refundable = st.selectbox("Is Refundable?", [0, 1])
    is_non_stop = st.selectbox("Is Non-Stop?", [0, 1])
    days_before_flight = st.number_input("Days Before Flight", min_value=0)
    day = st.number_input("Day of Month of Flight", min_value=1, max_value=31)
    num_legs = st.number_input("Number of Legs", min_value=1, max_value=4)
    all_same = st.selectbox("Are All Legs on the Same Airline?", [0, 1])
    distance = st.number_input("Distance (miles)", value=0.0)
    departure_hour = st.number_input("Departure Hour", min_value=0, max_value=23)
    departure_dow_idx = st.number_input("Departure Day of Week Index (0=Sunday, 6=Saturday)", value=0.0)
    starting_pop = st.number_input("Starting Airport Population", value=0.0)
    destination_pop = st.number_input("Destination Airport Population", value=0.0)
    starting_airport = st.text_input("Starting Airport")
    destination_airport = st.text_input("Ending Airport")
    airline_name = st.text_input("Airline Name")

    # Button to make prediction
    if st.button('Predict'):
        # Create a DataFrame from the input features
        features = pd.DataFrame({
            'is_basic_economy': [is_basic_economy],
            'is_refundable': [is_refundable],
            'is_non_stop': [is_non_stop],
            'days_before_flight': [days_before_flight],
            'day': [day],
            'num_legs': [num_legs],
            'all_same': [all_same],
            'distance': [distance],
            'departure_hour': [departure_hour],
            'departure_dow_idx': [departure_dow_idx],
            'starting_pop': [starting_pop],
            'destination_pop': [destination_pop],
            'starting_airport': [starting_airport],
            'destination_airport': [destination_airport],
            'airline_name': [airline_name]
        })

        model = load_model()
        prediction = model.predict(features)
        st.write(f"Predicted Output: {prediction[0]}")

if __name__ == "__main__":
    main()


2024-04-15 22:52:20.613 
  command:

    streamlit run /Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /Users/jordan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py 